In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\prana\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
bucket = "mlsagemakerbucket112"
sm_boto3 = boto3.client('sagemaker')
sess = sagemaker.Session()
region = sess.boto_session.region_name
print('Using bucket '+ bucket)

Using bucket mlsagemakerbucket112


In [3]:
df=pd.read_csv("mob_price_classification_train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
trainX,testX  = train_test_split(df,test_size = 0.1,random_state = 42)

In [5]:
trainX

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1872,1745,0,2.9,0,0,1,3,0.9,105,3,...,426,1629,1308,5,0,2,1,1,0,1
526,535,1,0.5,1,8,1,54,0.5,145,8,...,710,939,2674,14,8,10,1,0,0,2
173,1577,0,0.5,1,0,1,42,0.3,197,5,...,1045,1737,2060,19,6,12,1,0,0,2
720,1702,1,2.3,0,12,1,52,0.5,145,2,...,1397,1491,2501,16,12,4,1,0,0,3
414,707,0,2.1,1,2,0,25,0.8,131,3,...,495,574,3838,9,2,7,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,1975,1,1.9,1,2,0,31,0.9,151,1,...,775,1607,3022,13,5,19,0,0,1,3
1294,589,1,0.5,0,1,1,59,0.7,146,8,...,759,1858,362,16,10,6,1,1,1,0
860,1829,1,0.5,0,0,1,15,0.4,160,5,...,729,1267,2080,16,11,12,1,0,1,2
1459,1927,0,0.9,1,3,0,11,0.4,190,8,...,491,1506,2916,16,11,18,0,1,1,3


In [6]:
trainX.to_csv('train_v1.csv',index = False)
testX.to_csv('test_v1.csv',index = False)

In [8]:
#uploading csv to s3 bucket
path = "sagemaker/mobile_price_classification/sklearncontainer"
trainpath = sess.upload_data(path = 'train_v1.csv',bucket=bucket,key_prefix=path)
testpath = sess.upload_data(path = 'test_v1.csv',bucket =bucket, key_prefix=path)
print(trainpath)
print(testpath)

s3://mlsagemakerbucket112/sagemaker/mobile_price_classification/sklearncontainer/train_v1.csv
s3://mlsagemakerbucket112/sagemaker/mobile_price_classification/sklearncontainer/test_v1.csv


In [ ]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd


def model_fn(model_dir):
    clf  = joblib.load(os.path.join(model_dir,'model.joblib'))
    return clf

if __name__=='__main__':
    print("[Info] Extracting arguments")
    parser=argparse.ArgumentParser()

    ## Hyperparameter
    parser.add_argument("--n_estimators",type=int,default=100)
    parser.add_argument("--random_state",type=int,default=0)

    ### Data,model,and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train_v1.csv")
    parser.add_argument("--test-file", type=str, default="test_v1.csv")

    args, _ =parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()


    print("Training RandomForest Model ....")
    print()
    model=RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state,
                                 verbose=2,n_jobs=1)
    
    model.fit(X_train,y_train)

    print()

    model_path=os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model,model_path)

    print("Model saved at" + model_path)

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)






Writing script.py


In [14]:
from sagemaker.sklearn.estimator import SKLearn
FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point = 'script.py',
    role = "arn:aws:iam::597743362495:role/sagemakerproj1",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators":100,
        "random_state":0
    },
    use_spot_instance=True,
    max_run=3600
)

In [15]:
sklearn_estimator.fit({"train":trainpath,"test":testpath},wait = True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2025-06-30-16-49-56-075


2025-06-30 16:50:07 Starting - Starting the training job...
2025-06-30 16:50:42 Downloading - Downloading input data...
2025-06-30 16:51:06 Downloading - Downloading the training image.....2025-06-30 16:51:56,556 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-06-30 16:51:56,559 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-30 16:51:56,599 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-06-30 16:51:56,758 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-30 16:51:56,770 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-30 16:51:56,781 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-06-30 16:51:56,790 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 sklearn_estimator.fit({"train":trainpath,"test":testpath},wait = True)                       │
│   2                                                                                              │
│                                                                                                  │
│ e:\MLOPS\awssagemaker_proj\awssagemaker_venv\lib\site-packages\sagemaker\telemetry\telemetry_log │
│ ging.py:167 in wrapper                                                                           │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ e:\MLOPS\awssagemaker_proj\awssagemaker_venv\lib\site-packages\sagemaker\telemetry\telemetry_log │
│ ging.py:138 in wrapper                                                                           │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ e:\MLOPS\awssagemaker_proj\awssagemaker_venv\lib\site-packages\sagemaker\workflow\pipeline_conte │
│ xt.py:346 in wrapper                                                                             │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ e:\MLOPS\awssagemaker_proj\awssagemaker_venv\lib\site-packa